In [1]:
# import the packges
#import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import seaborn as sns

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lige2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# display the claim_final.csv
claim = pd.read_csv('Claim_final.csv')
claim.head()

,Unnamed: 0,CLM_NBR,NBR_OF_CLMT,BI_CLMT_CNT,COV_RGST_DT,CLAIM_EXPENSE_EST_AMT,CLAIM_INDEMNITY_EST_AMT,POL_NBR,POLICY_STATE,PRI_BTH_DT
0,0,YB85Q6L,1,2,5/29/2018,903,1522,991120685,NY,6/22/1960
1,1,VUC6B3X,1,2,11/28/2016,0,10093,993950729,FL,12/1/1944
2,6,EKICR4W,1,1,5/10/2018,156,75055,998685688,SC,6/10/1972
3,12,Y9TIRJH,2,1,12/11/2019,0,5576,994335003,FL,3/20/1988
4,16,YZKNJDD,1,3,7/7/2018,9129,13133,996569175,FL,9/15/1949


In [3]:
#take a look at the data on claim
print(claim.shape)
print(list(claim.columns))

(1572, 10)
['Unnamed: 0', 'CLM_NBR', 'NBR_OF_CLMT', 'BI_CLMT_CNT', 'COV_RGST_DT', 'CLAIM_EXPENSE_EST_AMT', 'CLAIM_INDEMNITY_EST_AMT', 'POL_NBR', 'POLICY_STATE', 'PRI_BTH_DT']


In [4]:
# remove columns permanently from claim_final.csv
claim.drop(["POL_NBR", "POLICY_STATE"], axis=1, inplace = True)
claim.head()

,Unnamed: 0,CLM_NBR,NBR_OF_CLMT,BI_CLMT_CNT,COV_RGST_DT,CLAIM_EXPENSE_EST_AMT,CLAIM_INDEMNITY_EST_AMT,PRI_BTH_DT
0,0,YB85Q6L,1,2,5/29/2018,903,1522,6/22/1960
1,1,VUC6B3X,1,2,11/28/2016,0,10093,12/1/1944
2,6,EKICR4W,1,1,5/10/2018,156,75055,6/10/1972
3,12,Y9TIRJH,2,1,12/11/2019,0,5576,3/20/1988
4,16,YZKNJDD,1,3,7/7/2018,9129,13133,9/15/1949


In [5]:
#add CLAIM_EXPENSE_EST_AMT and CLAIM_INDEMNITY_EST_AMT to get the total cost
#claim["TOTAL_COST"] = claim["CLAIM_EXPENSE_EST_AMT"] + claim["CLAIM_INDEMNITY_EST_AMT"]
#claim.head()

In [6]:
# get CLAIM_INDEMNITY_EST_PER
claim["CLAIM_INDEMNITY_EST_PER"] = (claim["CLAIM_INDEMNITY_EST_AMT"] / (claim["CLAIM_EXPENSE_EST_AMT"] + claim["CLAIM_INDEMNITY_EST_AMT"])) * 100
claim['CLAIM_INDEMNITY_EST_PER'] = claim['CLAIM_INDEMNITY_EST_PER'].fillna(0)
claim.head()

,Unnamed: 0,CLM_NBR,NBR_OF_CLMT,BI_CLMT_CNT,COV_RGST_DT,CLAIM_EXPENSE_EST_AMT,CLAIM_INDEMNITY_EST_AMT,PRI_BTH_DT,CLAIM_INDEMNITY_EST_PER
0,0,YB85Q6L,1,2,5/29/2018,903,1522,6/22/1960,62.762887
1,1,VUC6B3X,1,2,11/28/2016,0,10093,12/1/1944,100.000000
2,6,EKICR4W,1,1,5/10/2018,156,75055,6/10/1972,99.792584
3,12,Y9TIRJH,2,1,12/11/2019,0,5576,3/20/1988,100.000000
4,16,YZKNJDD,1,3,7/7/2018,9129,13133,9/15/1949,58.992903


In [7]:
claim.describe()

,Unnamed: 0,NBR_OF_CLMT,BI_CLMT_CNT,CLAIM_EXPENSE_EST_AMT,CLAIM_INDEMNITY_EST_AMT,CLAIM_INDEMNITY_EST_PER
count,1572.000000,1572.000000,1572.000000,1572.000000,1572.000000,1572.000000
mean,4275.779898,1.419847,1.776081,1013.190840,21820.115140,85.739744
std,2510.541155,0.838636,1.216056,4038.532404,42294.879636,32.067663
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,2009.000000,1.000000,1.000000,0.000000,1938.750000,96.169433
50%,4336.000000,1.000000,1.000000,0.000000,8847.000000,100.000000
75%,6400.000000,2.000000,2.000000,8.250000,24624.500000,100.000000
max,8641.000000,8.000000,10.000000,60000.000000,534429.000000,100.000000


In [8]:
# get CLAIM_EXPENSE_EST_PER
claim["CLAIM_EXPENSE_EST_PER"] = (claim["CLAIM_EXPENSE_EST_AMT"] / (claim["CLAIM_EXPENSE_EST_AMT"] + claim["CLAIM_INDEMNITY_EST_AMT"])) * 100
claim['CLAIM_EXPENSE_EST_PER'] = claim['CLAIM_EXPENSE_EST_PER'].fillna(0)
claim.head()

,Unnamed: 0,CLM_NBR,NBR_OF_CLMT,BI_CLMT_CNT,COV_RGST_DT,CLAIM_EXPENSE_EST_AMT,CLAIM_INDEMNITY_EST_AMT,PRI_BTH_DT,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER
0,0,YB85Q6L,1,2,5/29/2018,903,1522,6/22/1960,62.762887,37.237113
1,1,VUC6B3X,1,2,11/28/2016,0,10093,12/1/1944,100.000000,0.000000
2,6,EKICR4W,1,1,5/10/2018,156,75055,6/10/1972,99.792584,0.207416
3,12,Y9TIRJH,2,1,12/11/2019,0,5576,3/20/1988,100.000000,0.000000
4,16,YZKNJDD,1,3,7/7/2018,9129,13133,9/15/1949,58.992903,41.007097


In [9]:
claim.describe()

,Unnamed: 0,NBR_OF_CLMT,BI_CLMT_CNT,CLAIM_EXPENSE_EST_AMT,CLAIM_INDEMNITY_EST_AMT,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER
count,1572.000000,1572.000000,1572.000000,1572.000000,1572.000000,1572.000000,1572.000000
mean,4275.779898,1.419847,1.776081,1013.190840,21820.115140,85.739744,4.400205
std,2510.541155,0.838636,1.216056,4038.532404,42294.879636,32.067663,15.027058
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2009.000000,1.000000,1.000000,0.000000,1938.750000,96.169433,0.000000
50%,4336.000000,1.000000,1.000000,0.000000,8847.000000,100.000000,0.000000
75%,6400.000000,2.000000,2.000000,8.250000,24624.500000,100.000000,0.118564
max,8641.000000,8.000000,10.000000,60000.000000,534429.000000,100.000000,100.000000


In [10]:
# remove columns permanently from claim_final.csv
claim.drop(["CLAIM_EXPENSE_EST_AMT", "CLAIM_INDEMNITY_EST_AMT"], axis=1, inplace = True)
claim.head()

,Unnamed: 0,CLM_NBR,NBR_OF_CLMT,BI_CLMT_CNT,COV_RGST_DT,PRI_BTH_DT,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER
0,0,YB85Q6L,1,2,5/29/2018,6/22/1960,62.762887,37.237113
1,1,VUC6B3X,1,2,11/28/2016,12/1/1944,100.000000,0.000000
2,6,EKICR4W,1,1,5/10/2018,6/10/1972,99.792584,0.207416
3,12,Y9TIRJH,2,1,12/11/2019,3/20/1988,100.000000,0.000000
4,16,YZKNJDD,1,3,7/7/2018,9/15/1949,58.992903,41.007097


In [11]:
#show the relationship between NBR_OF_CLMT and BI_CLMT_CNT and make it a category
claim['CLAIMANT_NUM'] = claim.apply(lambda row: 0 if row['NBR_OF_CLMT'] > row['BI_CLMT_CNT'] else 1 if row['NBR_OF_CLMT'] == row['BI_CLMT_CNT'] else 2, axis=1)
claim.head()

,Unnamed: 0,CLM_NBR,NBR_OF_CLMT,BI_CLMT_CNT,COV_RGST_DT,PRI_BTH_DT,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER,CLAIMANT_NUM
0,0,YB85Q6L,1,2,5/29/2018,6/22/1960,62.762887,37.237113,2
1,1,VUC6B3X,1,2,11/28/2016,12/1/1944,100.000000,0.000000,2
2,6,EKICR4W,1,1,5/10/2018,6/10/1972,99.792584,0.207416,1
3,12,Y9TIRJH,2,1,12/11/2019,3/20/1988,100.000000,0.000000,0
4,16,YZKNJDD,1,3,7/7/2018,9/15/1949,58.992903,41.007097,2


In [12]:
# remove columns permanently from claim_final.csv
claim.drop(["NBR_OF_CLMT", "BI_CLMT_CNT"], axis=1, inplace = True)
claim.head()

,Unnamed: 0,CLM_NBR,COV_RGST_DT,PRI_BTH_DT,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER,CLAIMANT_NUM
0,0,YB85Q6L,5/29/2018,6/22/1960,62.762887,37.237113,2
1,1,VUC6B3X,11/28/2016,12/1/1944,100.000000,0.000000,2
2,6,EKICR4W,5/10/2018,6/10/1972,99.792584,0.207416,1
3,12,Y9TIRJH,12/11/2019,3/20/1988,100.000000,0.000000,0
4,16,YZKNJDD,7/7/2018,9/15/1949,58.992903,41.007097,2


In [13]:
# display the claim_final.csv
claimant = pd.read_csv('Claimant_final.csv')
claimant.head()

,Unnamed: 0,CLM_NBR,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,VEH_COLR_TXT,VEH_DAM_TXT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,VEH_TYPE,VEH_PRIMARY_PT_OF_DAMAGE,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT
0,0,YB85Q6L,Multi Veh - Unclear Liab,Closed with Payment,1,BURGANDY,"D/S FRONT FENDER, D",F-350 SUPE,2003,Automobile,Driver Front Corner,None,Below 10 mph
1,1,VUC6B3X,Multi Veh - Unclear Liab,Closed with Payment,2,BLACK,UNK REAR DMG,SONATA HYB,2016,Automobile,Complete Rear End,None,Parked or Stopped
2,6,EKICR4W,Multi Veh - Unclear Liab,Closed with Payment,1,BLACK,POI: D/S DOOR WON'T,200 LX,2016,Automobile,Driver Center,None,10-20 mph
3,12,Y9TIRJH,Multi Veh - Clear Liab,Closed without Payment,1,SILVER,NONE,COROLLA CE,2007,Automobile,No Damage,None,21-50 mph
4,16,YZKNJDD,Multi Veh - Unclear Liab,Closed with Payment,2,BLUE,PRIMARY POINT OF DA,F-150 SUPE,2009,Automobile,Complete Front End,One,Unknown


In [14]:
# remove columns permanently from claimant_final.csv
claimant.drop(["VEH_COLR_TXT", "VEH_TYPE"], axis=1, inplace = True)
claimant.head()

,Unnamed: 0,CLM_NBR,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,VEH_DAM_TXT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,VEH_PRIMARY_PT_OF_DAMAGE,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT
0,0,YB85Q6L,Multi Veh - Unclear Liab,Closed with Payment,1,"D/S FRONT FENDER, D",F-350 SUPE,2003,Driver Front Corner,None,Below 10 mph
1,1,VUC6B3X,Multi Veh - Unclear Liab,Closed with Payment,2,UNK REAR DMG,SONATA HYB,2016,Complete Rear End,None,Parked or Stopped
2,6,EKICR4W,Multi Veh - Unclear Liab,Closed with Payment,1,POI: D/S DOOR WON'T,200 LX,2016,Driver Center,None,10-20 mph
3,12,Y9TIRJH,Multi Veh - Clear Liab,Closed without Payment,1,NONE,COROLLA CE,2007,No Damage,None,21-50 mph
4,16,YZKNJDD,Multi Veh - Unclear Liab,Closed with Payment,2,PRIMARY POINT OF DA,F-150 SUPE,2009,Complete Front End,One,Unknown


In [15]:
#take a look at AUTO_ACCIDENT_DESC
unique1 = claimant['AUTO_ACCIDENT_DESC'].unique()
print(unique1)

['Multi Veh - Unclear Liab' 'Multi Veh - Clear Liab' 'Single Veh']


In [16]:
# replace Multi Veh - Unclear Liab to 0, Multi Veh - Clear Liab to 1, Single Veh to 2
mapping = {'Multi Veh - Unclear Liab': 0, 'Multi Veh - Clear Liab': 1, 'Single Veh': 2}
claimant['AUTO_ACCIDENT_DESC'] = claimant['AUTO_ACCIDENT_DESC'].replace(mapping)
claimant.head()

,Unnamed: 0,CLM_NBR,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,VEH_DAM_TXT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,VEH_PRIMARY_PT_OF_DAMAGE,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT
0,0,YB85Q6L,0,Closed with Payment,1,"D/S FRONT FENDER, D",F-350 SUPE,2003,Driver Front Corner,None,Below 10 mph
1,1,VUC6B3X,0,Closed with Payment,2,UNK REAR DMG,SONATA HYB,2016,Complete Rear End,None,Parked or Stopped
2,6,EKICR4W,0,Closed with Payment,1,POI: D/S DOOR WON'T,200 LX,2016,Driver Center,None,10-20 mph
3,12,Y9TIRJH,1,Closed without Payment,1,NONE,COROLLA CE,2007,No Damage,None,21-50 mph
4,16,YZKNJDD,0,Closed with Payment,2,PRIMARY POINT OF DA,F-150 SUPE,2009,Complete Front End,One,Unknown


In [17]:
claimant['VEH_PRIMARY_PT_OF_DAMAGE'] = claimant['VEH_PRIMARY_PT_OF_DAMAGE'].str.upper()
claimant.head()

,Unnamed: 0,CLM_NBR,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,VEH_DAM_TXT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,VEH_PRIMARY_PT_OF_DAMAGE,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT
0,0,YB85Q6L,0,Closed with Payment,1,"D/S FRONT FENDER, D",F-350 SUPE,2003,DRIVER FRONT CORNER,None,Below 10 mph
1,1,VUC6B3X,0,Closed with Payment,2,UNK REAR DMG,SONATA HYB,2016,COMPLETE REAR END,None,Parked or Stopped
2,6,EKICR4W,0,Closed with Payment,1,POI: D/S DOOR WON'T,200 LX,2016,DRIVER CENTER,None,10-20 mph
3,12,Y9TIRJH,1,Closed without Payment,1,NONE,COROLLA CE,2007,NO DAMAGE,None,21-50 mph
4,16,YZKNJDD,0,Closed with Payment,2,PRIMARY POINT OF DA,F-150 SUPE,2009,COMPLETE FRONT END,One,Unknown


In [18]:
#In VEH_DAM_TXT,  replace 'NONE' with 'NO DAMAGE'
claimant['VEH_DAM_TXT'] = claimant['VEH_DAM_TXT'].replace('NONE', 'NO DAMAGE')
# change DS$D/S to DRIVER SIDE
claimant['VEH_DAM_TXT'] = claimant['VEH_DAM_TXT'].str.replace(r'\b(D/S|DS)\b', 'DRIVER SIDE', regex=True)
# change PS$P/S to PASSENGER SIDE
claimant['VEH_DAM_TXT'] = claimant['VEH_DAM_TXT'].str.replace(r'\b(P/S|PS)\b', 'PASSENGER SIDE', regex=True)
claimant.head()

,Unnamed: 0,CLM_NBR,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,VEH_DAM_TXT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,VEH_PRIMARY_PT_OF_DAMAGE,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT
0,0,YB85Q6L,0,Closed with Payment,1,"DRIVER SIDE FRONT FENDER, D",F-350 SUPE,2003,DRIVER FRONT CORNER,None,Below 10 mph
1,1,VUC6B3X,0,Closed with Payment,2,UNK REAR DMG,SONATA HYB,2016,COMPLETE REAR END,None,Parked or Stopped
2,6,EKICR4W,0,Closed with Payment,1,POI: DRIVER SIDE DOOR WON'T,200 LX,2016,DRIVER CENTER,None,10-20 mph
3,12,Y9TIRJH,1,Closed without Payment,1,NO DAMAGE,COROLLA CE,2007,NO DAMAGE,None,21-50 mph
4,16,YZKNJDD,0,Closed with Payment,2,PRIMARY POINT OF DA,F-150 SUPE,2009,COMPLETE FRONT END,One,Unknown


In [19]:
# take a look at CLMT_VEH_MDL_NM
unique2 = claimant['CLMT_VEH_MDL_NM'].unique()
print(unique2)

['F-350 SUPE' 'SONATA HYB' '200 LX' 'COROLLA CE' 'F-150 SUPE' 'OUTBACK 3.'
 'SRX PREMIU' 'TC' 'YUKON SLE/' 'LEGACY 2.5' 'FOCUS ZX4' 'GRAND CHER'
 'ENDEAVOR X' 'ACCORD SPO' 'MATRIX BAS' 'RANGER' 'IMPALA LS' 'RAV4'
 'RAV 4 LIMITED' 'PRIUS' 'CAMRY L/LE' '320I' 'FORESTER 2' 'AVALANCHE'
 'CIVIC LX' 'MAXIMA SE/' 'CENTURY CU' 'RANGER SUP' 'ALTIMA' 'CTS LUXURY'
 'XTERRA X/S' 'CIVIC EX' 'EXPEDITION' 'AVEO BASE/' 'JETTA' 'CRV' 'ROGUE'
 'RAM PICKUP' 'CHALLENGER' 'FUSION SE' 'JETTA S/SE' 'ACCORD EX' 'COMPASS'
 'JETTA BASE' 'TACOMA DOU' 'UNKNOWN' 'FORESTER' 'CAMRY CE/L' 'DART SXT/D'
 'ODYSSEY EX' 'LEGACY BRI' 'A7' 'ALTIMA 2.5' '500 POP' 'HR-V LX AW'
 'TACOMA BAS' 'COROLLA' 'RAV4 XLE' 'TUNDRA DOU' 'CTS' 'PARK AVENU'
 'LUCERNE CX' 'ALTIMA BAS' 'RAV4 LE' 'PASSAT GLS' 'PICKUP 150'
 'PICKUP 350' 'ACCORD LX' '335I' 'FRONTIER X' 'ACCENT GLS' '4RUNNER SR'
 'ACADIA SLT' 'EXPLORER S' '535XI AWD' 'MAZDA3 I T' 'SENTRA BAS'
 'SILVERADO' 'MDX AWD (S' '626 LX/626' 'RIO LX' 'LEGACY O' 'SENTRA'
 'FOCUS SE/S' 'SIERR

In [20]:
claimant['CLMT_VEH_MDL_NM'], _ = pd.factorize(claimant['CLMT_VEH_MDL_NM'])
claimant.head()

,Unnamed: 0,CLM_NBR,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,VEH_DAM_TXT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,VEH_PRIMARY_PT_OF_DAMAGE,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT
0,0,YB85Q6L,0,Closed with Payment,1,"DRIVER SIDE FRONT FENDER, D",0,2003,DRIVER FRONT CORNER,None,Below 10 mph
1,1,VUC6B3X,0,Closed with Payment,2,UNK REAR DMG,1,2016,COMPLETE REAR END,None,Parked or Stopped
2,6,EKICR4W,0,Closed with Payment,1,POI: DRIVER SIDE DOOR WON'T,2,2016,DRIVER CENTER,None,10-20 mph
3,12,Y9TIRJH,1,Closed without Payment,1,NO DAMAGE,3,2007,NO DAMAGE,None,21-50 mph
4,16,YZKNJDD,0,Closed with Payment,2,PRIMARY POINT OF DA,4,2009,COMPLETE FRONT END,One,Unknown


In [21]:
#def text_to_words(text):
    # Tokenize the text into a list of words
    #words = word_tokenize(text)
    #return words

In [22]:
#claimant['VEH_DAM_WORD'] =claimant['VEH_DAM_TXT'].apply(text_to_words)
#claimant.head()

In [23]:
#claimant['VEH_PRIMARY_PT_OF_DAMAGE_WORD'] =claimant['VEH_PRIMARY_PT_OF_DAMAGE'].apply(text_to_words)
#claimant.head()

In [24]:
def check_common_words(text1, text2):
    # Tokenize the text descriptions into sets of words
    words1 = set(word_tokenize(text1))
    words2 = set(word_tokenize(text2))
    
    # Check if there is any intersection between the two sets
    if len(words1.intersection(words2)) >= 1:
        return 1
    else:
        return 0

In [25]:
claimant['Common Words'] = claimant.apply(lambda x: check_common_words(x['VEH_DAM_TXT'], x['VEH_PRIMARY_PT_OF_DAMAGE']), axis=1)
claimant.head()

,Unnamed: 0,CLM_NBR,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,VEH_DAM_TXT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,VEH_PRIMARY_PT_OF_DAMAGE,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words
0,0,YB85Q6L,0,Closed with Payment,1,"DRIVER SIDE FRONT FENDER, D",0,2003,DRIVER FRONT CORNER,None,Below 10 mph,1
1,1,VUC6B3X,0,Closed with Payment,2,UNK REAR DMG,1,2016,COMPLETE REAR END,None,Parked or Stopped,1
2,6,EKICR4W,0,Closed with Payment,1,POI: DRIVER SIDE DOOR WON'T,2,2016,DRIVER CENTER,None,10-20 mph,1
3,12,Y9TIRJH,1,Closed without Payment,1,NO DAMAGE,3,2007,NO DAMAGE,None,21-50 mph,1
4,16,YZKNJDD,0,Closed with Payment,2,PRIMARY POINT OF DA,4,2009,COMPLETE FRONT END,One,Unknown,0


In [26]:
#drop VEH_DAM_TXT and VEH_PRIMARY_PT_OF_DAMAGE
claimant.drop(["VEH_DAM_TXT", "VEH_PRIMARY_PT_OF_DAMAGE"], axis=1, inplace = True)
claimant.head()

,Unnamed: 0,CLM_NBR,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words
0,0,YB85Q6L,0,Closed with Payment,1,0,2003,None,Below 10 mph,1
1,1,VUC6B3X,0,Closed with Payment,2,1,2016,None,Parked or Stopped,1
2,6,EKICR4W,0,Closed with Payment,1,2,2016,None,10-20 mph,1
3,12,Y9TIRJH,1,Closed without Payment,1,3,2007,None,21-50 mph,1
4,16,YZKNJDD,0,Closed with Payment,2,4,2009,One,Unknown,0


In [27]:
#take a look at VEH_PRIMARY_PT_OF_DAMAGE
#unique3 = claimant['VEH_PRIMARY_PT_OF_DAMAGE'].unique()
#print(unique3)

In [28]:
#remake VEH_PRIMARY_PT_OF_DAMAGE
#mapping = {'Driver Front Corner': 0, 'Complete Rear End': 1, 'Driver Center': 2,
          #'No Damage': 3, 'Complete Front End':4, 'Driver Rear Corner':5, 'Complete Passenger Side':6,
           #'All Over': 7, 'Passenger Front Corner': 8, 'Complete Driver Side':9, 'Passenger Center': 10,
          #'Passenger Rear Corner': 11, 'Roof': 12}
#claimant['VEH_PRIMARY_PT_OF_DAMAGE'] = claimant['VEH_PRIMARY_PT_OF_DAMAGE'].replace(mapping)
#claimant.head()

In [29]:
#take a look at AIRBAG_DEPLOYED
unique4 = claimant['AIRBAG_DEPLOYED'].unique()
print(unique4)

['None' 'One' 'More than one' 'Unknown']


In [30]:
# remake AIRBAG_DEPLOYED
mapping = {'None': 0, 'One': 1, 'More than one': 2, 'Unknown': 3}
claimant['AIRBAG_DEPLOYED'] = claimant['AIRBAG_DEPLOYED'].replace(mapping)
claimant.head()

,Unnamed: 0,CLM_NBR,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words
0,0,YB85Q6L,0,Closed with Payment,1,0,2003,0,Below 10 mph,1
1,1,VUC6B3X,0,Closed with Payment,2,1,2016,0,Parked or Stopped,1
2,6,EKICR4W,0,Closed with Payment,1,2,2016,0,10-20 mph,1
3,12,Y9TIRJH,1,Closed without Payment,1,3,2007,0,21-50 mph,1
4,16,YZKNJDD,0,Closed with Payment,2,4,2009,1,Unknown,0


In [31]:
#take a look at VEH_SPEED_AT_IMPACT
unique4 = claimant['VEH_SPEED_AT_IMPACT'].unique()
print(unique4)

['Below 10 mph' 'Parked or Stopped' '10-20 mph' '21-50 mph' 'Unknown'
 'Over 50 mph']


In [32]:
# remake VEH_SPEED_AT_IMPACT
mapping = {'Parked or Stopped': 0, 'Below 10 mph': 1, '10-20 mph': 2, '21-50 mph': 3, 'Over 50 mph': 4, 'Unknown': 5}
claimant['VEH_SPEED_AT_IMPACT'] = claimant['VEH_SPEED_AT_IMPACT'].replace(mapping)
claimant.head()

,Unnamed: 0,CLM_NBR,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words
0,0,YB85Q6L,0,Closed with Payment,1,0,2003,0,1,1
1,1,VUC6B3X,0,Closed with Payment,2,1,2016,0,0,1
2,6,EKICR4W,0,Closed with Payment,1,2,2016,0,2,1
3,12,Y9TIRJH,1,Closed without Payment,1,3,2007,0,3,1
4,16,YZKNJDD,0,Closed with Payment,2,4,2009,1,5,0


In [33]:
#take a look at CLAIMANT_STATUS
unique5 = claimant['CLAIMANT_STATUS'].unique()
print(unique5)

['Closed with Payment' 'Closed without Payment' 'Closed to Subrogation']


In [34]:
# remake CLAIMANT_STATUS
mapping = {'Closed with Payment': 0, 'Closed without Payment': 1, 'Closed to Subrogation': 2}
claimant['CLAIMANT_STATUS'] = claimant['CLAIMANT_STATUS'].replace(mapping)
claimant.head()

,Unnamed: 0,CLM_NBR,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words
0,0,YB85Q6L,0,0,1,0,2003,0,1,1
1,1,VUC6B3X,0,0,2,1,2016,0,0,1
2,6,EKICR4W,0,0,1,2,2016,0,2,1
3,12,Y9TIRJH,1,1,1,3,2007,0,3,1
4,16,YZKNJDD,0,0,2,4,2009,1,5,0


In [35]:
merged = pd.merge(claim, claimant)
merged.head()

,Unnamed: 0,CLM_NBR,COV_RGST_DT,PRI_BTH_DT,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER,CLAIMANT_NUM,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words
0,0,YB85Q6L,5/29/2018,6/22/1960,62.762887,37.237113,2,0,0,1,0,2003,0,1,1
1,1,VUC6B3X,11/28/2016,12/1/1944,100.000000,0.000000,2,0,0,2,1,2016,0,0,1
2,6,EKICR4W,5/10/2018,6/10/1972,99.792584,0.207416,1,0,0,1,2,2016,0,2,1
3,12,Y9TIRJH,12/11/2019,3/20/1988,100.000000,0.000000,0,1,1,1,3,2007,0,3,1
4,16,YZKNJDD,7/7/2018,9/15/1949,58.992903,41.007097,2,0,0,2,4,2009,1,5,0


In [36]:
# difference between COV_RGST_DT and PRI_BTH_DT
merged['COV_RGST_DT'] = pd.to_datetime(merged['COV_RGST_DT'])
merged['PRI_BTH_DT'] = pd.to_datetime(merged['PRI_BTH_DT'])
merged['DIFF_RGST__BRI'] = (merged['COV_RGST_DT'] - merged['PRI_BTH_DT']) // pd.Timedelta(days=365.25)
merged.head()

,Unnamed: 0,CLM_NBR,COV_RGST_DT,PRI_BTH_DT,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER,CLAIMANT_NUM,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words,DIFF_RGST__BRI
0,0,YB85Q6L,2018-05-29,1960-06-22,62.762887,37.237113,2,0,0,1,0,2003,0,1,1,57
1,1,VUC6B3X,2016-11-28,1944-12-01,100.000000,0.000000,2,0,0,2,1,2016,0,0,1,71
2,6,EKICR4W,2018-05-10,1972-06-10,99.792584,0.207416,1,0,0,1,2,2016,0,2,1,45
3,12,Y9TIRJH,2019-12-11,1988-03-20,100.000000,0.000000,0,1,1,1,3,2007,0,3,1,31
4,16,YZKNJDD,2018-07-07,1949-09-15,58.992903,41.007097,2,0,0,2,4,2009,1,5,0,68


In [37]:
# age of the policy holder
merged['PRI_AGE'] = (datetime.now() - merged['PRI_BTH_DT']) // pd.Timedelta(days=365.25)
merged.head()

,Unnamed: 0,CLM_NBR,COV_RGST_DT,PRI_BTH_DT,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER,CLAIMANT_NUM,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words,DIFF_RGST__BRI,PRI_AGE
0,0,YB85Q6L,2018-05-29,1960-06-22,62.762887,37.237113,2,0,0,1,0,2003,0,1,1,57,62
1,1,VUC6B3X,2016-11-28,1944-12-01,100.000000,0.000000,2,0,0,2,1,2016,0,0,1,71,78
2,6,EKICR4W,2018-05-10,1972-06-10,99.792584,0.207416,1,0,0,1,2,2016,0,2,1,45,50
3,12,Y9TIRJH,2019-12-11,1988-03-20,100.000000,0.000000,0,1,1,1,3,2007,0,3,1,31,34
4,16,YZKNJDD,2018-07-07,1949-09-15,58.992903,41.007097,2,0,0,2,4,2009,1,5,0,68,73


In [38]:
# get the year of the COV_RGST_DT and PRI_BTH_DT
merged['COV_RGST_YEAR'] = merged['COV_RGST_DT'].dt.year
merged['PRI_BTH_YEAR'] = merged['PRI_BTH_DT'].dt.year
merged.head()

,Unnamed: 0,CLM_NBR,COV_RGST_DT,PRI_BTH_DT,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER,CLAIMANT_NUM,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,CLMT_VEH_MDL_NM,CLMT_VEH_YR,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words,DIFF_RGST__BRI,PRI_AGE,COV_RGST_YEAR,PRI_BTH_YEAR
0,0,YB85Q6L,2018-05-29,1960-06-22,62.762887,37.237113,2,0,0,1,0,2003,0,1,1,57,62,2018,1960
1,1,VUC6B3X,2016-11-28,1944-12-01,100.000000,0.000000,2,0,0,2,1,2016,0,0,1,71,78,2016,1944
2,6,EKICR4W,2018-05-10,1972-06-10,99.792584,0.207416,1,0,0,1,2,2016,0,2,1,45,50,2018,1972
3,12,Y9TIRJH,2019-12-11,1988-03-20,100.000000,0.000000,0,1,1,1,3,2007,0,3,1,31,34,2019,1988
4,16,YZKNJDD,2018-07-07,1949-09-15,58.992903,41.007097,2,0,0,2,4,2009,1,5,0,68,73,2018,1949


In [39]:
merged.dtypes

Unnamed: 0                          int64
CLM_NBR                            object
COV_RGST_DT                datetime64[ns]
PRI_BTH_DT                 datetime64[ns]
CLAIM_INDEMNITY_EST_PER           float64
CLAIM_EXPENSE_EST_PER             float64
CLAIMANT_NUM                        int64
AUTO_ACCIDENT_DESC                  int64
CLAIMANT_STATUS                     int64
APPR_DAM_EST_REC_CNT                int64
CLMT_VEH_MDL_NM                     int64
CLMT_VEH_YR                         int64
AIRBAG_DEPLOYED                     int64
VEH_SPEED_AT_IMPACT                 int64
Common Words                        int64
DIFF_RGST__BRI                      int64
PRI_AGE                             int64
COV_RGST_YEAR                       int64
PRI_BTH_YEAR                        int64
dtype: object

In [40]:
# year differences between CLMT_VEH_YR and COV_RGST_YEAR, CLMT_VEH_YR and PRI_BTH_YEAR
merged['DIFF_VEH_RGST'] = merged['COV_RGST_YEAR'] - merged['CLMT_VEH_YR']
merged['DIFF_BTH_RGST'] = merged['COV_RGST_YEAR'] - merged['PRI_BTH_YEAR']
merged.head()

,Unnamed: 0,CLM_NBR,COV_RGST_DT,PRI_BTH_DT,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER,CLAIMANT_NUM,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,...,CLMT_VEH_YR,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words,DIFF_RGST__BRI,PRI_AGE,COV_RGST_YEAR,PRI_BTH_YEAR,DIFF_VEH_RGST,DIFF_BTH_RGST
0,0,YB85Q6L,2018-05-29,1960-06-22,62.762887,37.237113,2,0,0,1,...,2003,0,1,1,57,62,2018,1960,15,58
1,1,VUC6B3X,2016-11-28,1944-12-01,100.000000,0.000000,2,0,0,2,...,2016,0,0,1,71,78,2016,1944,0,72
2,6,EKICR4W,2018-05-10,1972-06-10,99.792584,0.207416,1,0,0,1,...,2016,0,2,1,45,50,2018,1972,2,46
3,12,Y9TIRJH,2019-12-11,1988-03-20,100.000000,0.000000,0,1,1,1,...,2007,0,3,1,31,34,2019,1988,12,31
4,16,YZKNJDD,2018-07-07,1949-09-15,58.992903,41.007097,2,0,0,2,...,2009,1,5,0,68,73,2018,1949,9,69


In [41]:
# drop the useless data
merged.drop(["COV_RGST_YEAR", "PRI_BTH_YEAR", "CLMT_VEH_YR", "COV_RGST_DT", "PRI_BTH_DT", "PRI_AGE", merged.columns[0]], axis=1, inplace = True)
#merged = merged.rename(columns={'Unnamed': 'ID'})
merged.head()

,CLM_NBR,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER,CLAIMANT_NUM,AUTO_ACCIDENT_DESC,CLAIMANT_STATUS,APPR_DAM_EST_REC_CNT,CLMT_VEH_MDL_NM,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words,DIFF_RGST__BRI,DIFF_VEH_RGST,DIFF_BTH_RGST
0,YB85Q6L,62.762887,37.237113,2,0,0,1,0,0,1,1,57,15,58
1,VUC6B3X,100.000000,0.000000,2,0,0,2,1,0,0,1,71,0,72
2,EKICR4W,99.792584,0.207416,1,0,0,1,2,0,2,1,45,2,46
3,Y9TIRJH,100.000000,0.000000,0,1,1,1,3,0,3,1,31,12,31
4,YZKNJDD,58.992903,41.007097,2,0,0,2,4,1,5,0,68,9,69


In [42]:
#extract data
#merged.to_csv('extracted_merged.csv', index=False)

In [43]:
# drop the useless data
merged.drop(["CLAIMANT_STATUS", "DIFF_BTH_RGST"], axis=1, inplace = True)
merged.head()

,CLM_NBR,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER,CLAIMANT_NUM,AUTO_ACCIDENT_DESC,APPR_DAM_EST_REC_CNT,CLMT_VEH_MDL_NM,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words,DIFF_RGST__BRI,DIFF_VEH_RGST
0,YB85Q6L,62.762887,37.237113,2,0,1,0,0,1,1,57,15
1,VUC6B3X,100.000000,0.000000,2,0,2,1,0,0,1,71,0
2,EKICR4W,99.792584,0.207416,1,0,1,2,0,2,1,45,2
3,Y9TIRJH,100.000000,0.000000,0,1,1,3,0,3,1,31,12
4,YZKNJDD,58.992903,41.007097,2,0,2,4,1,5,0,68,9


In [44]:
#extract data
#merged.to_csv('extracted_merged.csv', index=False)

In [45]:
#merged.drop(["CLM_NBR"], axis=1, inplace = True)
#merged.head()

In [46]:
corr_matrix = merged.corr()

In [47]:
corr_coefficient_1_2 = corr_matrix.iloc[1, 2]
print(corr_coefficient_1_2)

0.0842134366342615


In [48]:
corr_coefficient_1_3 = corr_matrix.iloc[1, 3]
print(corr_coefficient_1_3)

-0.036048485527183156


In [49]:
corr_coefficient_1_4 = corr_matrix.iloc[1, 4]
print(corr_coefficient_1_4)

-0.022675200498551807


In [50]:
corr_coefficient_1_5 = corr_matrix.iloc[1, 5]
print(corr_coefficient_1_5)

0.0038880039360697153


In [51]:
corr_coefficient_1_6 = corr_matrix.iloc[1, 6]
print(corr_coefficient_1_6)

-0.017018075120741317


In [52]:
corr_coefficient_1_7 = corr_matrix.iloc[1, 7]
print(corr_coefficient_1_7)

-0.01470226442354028


In [53]:
corr_coefficient_1_8 = corr_matrix.iloc[1, 8]
print(corr_coefficient_1_8)

0.051085287742787595


In [54]:
corr_coefficient_1_9 = corr_matrix.iloc[1, 9]
print(corr_coefficient_1_9)

0.018600893304989836


In [55]:
#column_10 = merged.iloc[:, 1].head()
#print(column_10)

In [56]:
categorical_features = ['CLAIMANT_NUM', 'AUTO_ACCIDENT_DESC', 'AIRBAG_DEPLOYED', 'VEH_SPEED_AT_IMPACT',"Common Words"]
continous_features = ['CLAIM_INDEMNITY_EST_PER', 'CLAIM_EXPENSE_EST_PER', 'APPR_DAM_EST_REC_CNT', 'DIFF_RGST__BRI', 'DIFF_VEH_RGST']

In [57]:
# remove columns permanently from claim_final.csv
merged.drop(["CLMT_VEH_MDL_NM"], axis=1, inplace = True)
merged.head()

,CLM_NBR,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER,CLAIMANT_NUM,AUTO_ACCIDENT_DESC,APPR_DAM_EST_REC_CNT,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words,DIFF_RGST__BRI,DIFF_VEH_RGST
0,YB85Q6L,62.762887,37.237113,2,0,1,0,1,1,57,15
1,VUC6B3X,100.000000,0.000000,2,0,2,0,0,1,71,0
2,EKICR4W,99.792584,0.207416,1,0,1,0,2,1,45,2
3,Y9TIRJH,100.000000,0.000000,0,1,1,0,3,1,31,12
4,YZKNJDD,58.992903,41.007097,2,0,2,1,5,0,68,9


In [58]:
merged[continous_features].describe()

,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER,APPR_DAM_EST_REC_CNT,DIFF_RGST__BRI,DIFF_VEH_RGST
count,1572.000000,1572.000000,1572.000000,1572.000000,1572.000000
mean,85.739744,4.400205,1.372774,42.360687,8.181934
std,32.067663,15.027058,0.861347,18.268534,12.769762
min,0.000000,0.000000,0.000000,-1.000000,-1.000000
25%,96.169433,0.000000,1.000000,28.000000,3.000000
50%,100.000000,0.000000,1.000000,41.000000,6.000000
75%,100.000000,0.118564,2.000000,56.000000,11.000000
max,100.000000,100.000000,7.000000,95.000000,119.000000


In [59]:
merged[categorical_features].describe()

,CLAIMANT_NUM,AUTO_ACCIDENT_DESC,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words
count,1572.000000,1572.000000,1572.000000,1572.000000,1572.000000
mean,1.160305,0.218830,0.571247,2.830153,0.589059
std,0.718760,0.606942,0.974212,1.911413,0.492161
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,1.000000,0.000000
50%,1.000000,0.000000,0.000000,3.000000,1.000000
75%,2.000000,0.000000,1.000000,5.000000,1.000000
max,2.000000,2.000000,3.000000,5.000000,1.000000


In [60]:
#X = merged.drop(["CLM_NBR"], axis=1)

In [61]:
#for col in categorical_features:
    #dummies = pd.get_dummies(merged[col], prefix=col)
    #merged = pd.concat([merged, dummies], axis=1)
    #merged.drop(col, axis=1, inplace=True)
#merged.head()

In [62]:
#extract data
#cluster_data.to_csv('fraud_data_version2.csv', index=False)

In [63]:
# calculate the Q1, Q3, and IQR for each numerical feature
q1 = merged[continous_features].quantile(0.25)
q3 = merged[continous_features].quantile(0.75)
iqr = q3 - q1

In [64]:
# calculate the lower and upper bounds for each numerical feature
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

In [65]:
numerical_outliers = ((merged[continous_features] < lower_bound) | (merged[continous_features] > upper_bound)).any(axis=1)

In [66]:
# identify outliers in the categorical feature
categorical_outliers = []
for feature in categorical_features:
    value_counts = merged[feature].value_counts()
    categories = value_counts.index
    expected_counts = len(merged) / len(categories)
    for category in categories:
        count = value_counts[category]
        if count < expected_counts * 0.5 or count > expected_counts * 1.5:
            categorical_outliers.append((feature, category))

In [67]:
# print the outliers
print("Numerical Outliers:")
print(merged[numerical_outliers])
print("Categorical Outliers:")
print(categorical_outliers)

Numerical Outliers:
      CLM_NBR  CLAIM_INDEMNITY_EST_PER  CLAIM_EXPENSE_EST_PER  CLAIMANT_NUM  \
0     YB85Q6L                62.762887              37.237113             2   
4     YZKNJDD                58.992903              41.007097             2   
6     CBH7XVD                95.810814               4.189186             2   
7     X8LZHG1                99.369066               0.630934             2   
10    V03XJKM                 0.000000               0.000000             1   
...       ...                      ...                    ...           ...   
1558  RY68U16                86.750522              13.249478             0   
1564  BMZ317K                 0.000000               0.000000             2   
1565  XI2T1SY                96.691836               3.308164             2   
1566  U4UZXVE                87.674305              12.325695             2   
1567  WFYQ1JA                91.229471               8.770529             1   

      AUTO_ACCIDENT_DESC  APPR_

In [68]:
Numerical_Outliers = merged[numerical_outliers]
Numerical_Outliers.head()

,CLM_NBR,CLAIM_INDEMNITY_EST_PER,CLAIM_EXPENSE_EST_PER,CLAIMANT_NUM,AUTO_ACCIDENT_DESC,APPR_DAM_EST_REC_CNT,AIRBAG_DEPLOYED,VEH_SPEED_AT_IMPACT,Common Words,DIFF_RGST__BRI,DIFF_VEH_RGST
0,YB85Q6L,62.762887,37.237113,2,0,1,0,1,1,57,15
4,YZKNJDD,58.992903,41.007097,2,0,2,1,5,0,68,9
6,CBH7XVD,95.810814,4.189186,2,0,2,0,5,0,11,2
7,X8LZHG1,99.369066,0.630934,2,0,1,2,3,0,29,10
10,V03XJKM,0.000000,0.000000,1,0,0,3,5,1,84,12


In [69]:
extracted_data = Numerical_Outliers[(Numerical_Outliers['AUTO_ACCIDENT_DESC'].isin([0, 1, 2])) & (Numerical_Outliers['AIRBAG_DEPLOYED'].isin([1, 3])) & (Numerical_Outliers['VEH_SPEED_AT_IMPACT'].isin([4, 5]))]

In [70]:
print(extracted_data)

      CLM_NBR  CLAIM_INDEMNITY_EST_PER  CLAIM_EXPENSE_EST_PER  CLAIMANT_NUM  \
4     YZKNJDD                58.992903              41.007097             2   
10    V03XJKM                 0.000000               0.000000             1   
46    F4I3DDN               100.000000               0.000000             2   
136   A20XOCS                95.469126               4.530874             2   
174   0E9YG83                 0.000000               0.000000             1   
...       ...                      ...                    ...           ...   
1438  8UPLXKO                 0.000000             100.000000             2   
1462  3ZNRL6A                97.417395               2.582605             2   
1489  MG87SJJ                91.689029               8.310971             2   
1496  1GURSI4                87.780899              12.219101             1   
1546  A966HIA                 0.000000               0.000000             2   

      AUTO_ACCIDENT_DESC  APPR_DAM_EST_REC_CNT  AIR

In [72]:
#extract data
extracted_data.to_csv('fraud data method2.csv', index=False)